## 5.1 template vs macro

In [1]:
#define MAX(x, y) (((x) > (y)) ? (x) : (y))

In [2]:
int a = 0;
int b = 1;
MAX(a++, b++)

(int) 2


In [3]:
#include <concepts>
#include <array>

In [4]:
template <typename T>
concept Comparable = requires(T a, T b){
    { a > b } -> std::same_as<bool>;
};

In [5]:
struct Foo{};

In [6]:
template<typename T>
T max(T a, T b) {return a > b ? a : b ;}

see compiler complaints below:

In [7]:
// max(Foo{}, Foo{});

see compiler complaints below:

In [8]:
template<typename T>
requires Comparable<T>
T comparable_max(T a, T b) {return a > b ? a : b ;}

In [9]:
// comparable_max(Foo{}, Foo{});

## 5.2 template class meta function

In [10]:
using Array5x4x3 = std::array<std::array<std::array<int, 3>, 4>, 5>;

In [11]:
template<typename T, size_t I, size_t ...Is>
struct Array{
    using type = std::array<typename Array<T, Is...>::type, I>;
};

// terminating condition:
template<typename T, size_t I>
struct Array<T, I>{
    using type = std::array<T, I>;
};


In [12]:
static_assert(std::is_same_v<Array<int, 5, 4, 3>::type, Array5x4x3>);

## 5.3 TypeList

In [13]:
template<typename ...Ts>
struct TypeList{};

using List = TypeList<int, double>;

In [14]:
using One = std::integral_constant<int, 1>;
constexpr auto one = One::value;
using Two = std::integral_constant<int, 2>;
constexpr auto two = Two::value;

In [15]:
using IntegralList = TypeList<One, Two>;

In [16]:
template <typename ...Ts>
struct TypeListBase {
    struct IsTypeList{};
    using type = TypeListBase;
    constexpr static size_t size = sizeof...(Ts);
    template<typename ...T> using append = TypeListBase<Ts..., T...>;
    template<typename ...T> using prepend = TypeListBase<T..., Ts...>;
    template<template<typename ...> typename T> using to = T<Ts...>;
};

template<typename TypeListBase>
concept TL = requires{
    typename TypeListBase::IsTypeList;
    typename TypeListBase::type;
};

In [17]:
using AList = TypeListBase<int, char>;

In [18]:
static_assert(TL<AList>);
static_assert(AList::size == 2);
static_assert(std::is_same_v<AList::prepend<double>, TypeListBase<double, int, char>>);
static_assert(std::is_same_v<AList::append<double>, TypeListBase<int, char, double>>);

more advanced functions:

In [19]:
template<TL In, template<typename> class F>
struct Map;
template<template<typename> class F, typename ...Ts>
struct Map<TypeListBase<Ts...>, F> : TypeListBase<typename F<Ts>::type...>{};

In [20]:
template<TL In, template<typename> class F>
using Map_t = typename Map<In, F>::type;

In [21]:
using LongList = TypeListBase<char, float, double, int, char>;

In [22]:
static_assert(std::is_same_v<Map_t<LongList, std::add_pointer>, TypeListBase<char*, float*, double*, int*, char*>>)